In [1]:
# Clone SEIR model repository
!git clone https://github.com/youyanggu/yyg-seir-simulator.git

# Clone COVID-19 data repository
!git clone https://github.com/CSSEGISandData/COVID-19.git


fatal: destination path 'yyg-seir-simulator' already exists and is not an empty directory.
fatal: destination path 'COVID-19' already exists and is not an empty directory.


In [2]:
import glob
import os
from IPython.display import display
import pandas as pd

# Load CSV file from the directory
data_path = "C:\\Users\\zooma\\ECE 227\\time_series_covid19_deaths_US.csv"
deaths_data = pd.read_csv(data_path)

# Convert the Date column to datetime
deaths_data['Date'] = pd.to_datetime(deaths_data['Date'])

# Filter the DataFrame by date range
start_date = pd.Timestamp('2020-02-01')
end_date = pd.Timestamp('2022-04-17')
deaths_data = deaths_data[(deaths_data['Date'] >= start_date) & (deaths_data['Date'] <= end_date)]

# Calculate daily deaths
deaths_data['Daily_Deaths'] = deaths_data['Deaths'].diff()

# Remove the first row with NaN values resulting from diff()
deaths_data = deaths_data.dropna(subset=['Daily_Deaths'])

# Apply smoothing using a 7-day rolling average
deaths_data['Smoothed_Daily_Deaths'] = deaths_data['Daily_Deaths'].rolling(window=7, min_periods=1).mean()

# Drop the original Deaths and Daily_Deaths columns
deaths_data = deaths_data.drop(columns=['Deaths', 'Daily_Deaths'])

# Rename the Smoothed_Daily_Deaths column to Deaths
deaths_data.rename(columns={'Smoothed_Daily_Deaths': 'Deaths'}, inplace=True)

# Create a DataFrame with the dates and smoothed daily deaths
smoothed_daily_deaths_df = deaths_data[['Date', 'Deaths']].reset_index(drop=True)

# # Display the smoothed DataFrame
# display(deaths_data.head())
# display(deaths_data.tail())

# # Save to a CSV file
# output_path = 'deaths_data.csv'
# deaths_data.to_csv(output_path, index=False)


,Date,Deaths
11,2020-02-02,0.0
12,2020-02-03,0.0
13,2020-02-04,0.0
14,2020-02-05,0.0
15,2020-02-06,0.0


,Date,Deaths
812,2022-04-13,532.714286
813,2022-04-14,487.285714
814,2022-04-15,462.142857
815,2022-04-16,445.714286
816,2022-04-17,443.142857


In [3]:
import numpy as np
import datetime
from itertools import product
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# Import the required modules from the SEIR model repository
import sys
sys.path.append('yyg-seir-simulator')

from region_model import RegionModel
from simulation import run

# Function to run a single simulation and calculate the loss
def run_simulation(params, deaths_data, param_keys):
    if params is None:
        raise ValueError("Received None for params")
    
    if any(p is None for p in params):
        raise ValueError(f"One of the params is None: {params}")
    
    # Initialize model parameters
    params_dict = {}
    for key, value in zip(param_keys, params):
        if isinstance(value, datetime.date):
            params_dict[key] = value
        else:
            params_dict[key] = float(value)
    
    # Run simulation
    first_date = datetime.date(2020, 2, 1)
    projection_create_date = datetime.date(2020, 5, 1)
    projection_end_date = datetime.date(2022, 4, 17)
    actual_deaths_smooth = deaths_data['Deaths'].values

    region_model = RegionModel(
        country_str='US',
        region_str='CA',
        subregion_str='',
        first_date=first_date,
        projection_create_date=projection_create_date,
        projection_end_date=projection_end_date,
        region_params={'population': 332000000},
        actual_deaths_smooth=actual_deaths_smooth,
        compute_hospitalizations=True
    )
    
    region_model.init_params(tuple(params_dict.items()))

    dates, infections, hospitalizations, deaths = run(region_model)

    # Ensure lengths match
    true_deaths = deaths_data['Deaths'].values[:len(deaths)]
    projected_deaths = deaths[:len(true_deaths)]

    loss = mean_squared_error(true_deaths, projected_deaths)
    # print(f"Loss: {loss}")
    
    return loss

# Split data into training and validation sets
train_data, val_data = train_test_split(deaths_data, test_size=0.2, shuffle=False)

# Helper function to perform grid search
def perform_grid_search(param_grid, fixed_params, param_keys, train_data):
    param_combinations = list(product(*param_grid.values()))
    
    best_params = None
    lowest_loss = float('inf')
    
    for params in tqdm(param_combinations, desc="Grid Search Progress"):
        try:
            combined_params = list(params) + list(fixed_params)
            loss = run_simulation(combined_params, deaths_data=train_data, param_keys=param_keys)
            if loss < lowest_loss:
                lowest_loss = loss
                best_params = combined_params
        except Exception as e:
            print(f"Error with parameters {combined_params}: {e}")
    
    return best_params, lowest_loss
    

In [4]:
# Fixed dates
INFLECTION_DAY = datetime.date(2020, 3, 18)
REOPEN_DATE = datetime.date(2020, 5, 20)

# Step 1: Grid search over the initial set of parameters
param_grid_step1 = {
    'INITIAL_R_0': np.linspace(0.8, 6.0, 52),
    'LOCKDOWN_R_0': np.linspace(0.3, 1.5, 12),
    'REOPEN_R': np.linspace(1.0, 1.5, 5),
    'POST_REOPEN_EQUILIBRIUM_R': np.linspace(0.85, 1.0, 15),
    'MORTALITY_RATE': np.linspace(0.005, 0.0125, 75),
}

fixed_params_step1 = [
    INFLECTION_DAY,  # Fixed INFLECTION_DAY
    REOPEN_DATE,  # Fixed REOPEN_DATE
    0,  # Fixed REOPEN_SHIFT_DAYS
    0.25,  # RATE_OF_INFLECTION
    1.0,  # LOCKDOWN_FATIGUE
    0.3,  # REOPEN_INFLECTION
    1.001,  # FALL_R_MULTIPLIER
    100,  # DAILY_IMPORTS
]

param_keys_step1 = list(param_grid_step1.keys()) + [
    'INFLECTION_DAY',
    'REOPEN_DATE',
    'REOPEN_SHIFT_DAYS',
    'RATE_OF_INFLECTION',
    'LOCKDOWN_FATIGUE',
    'REOPEN_INFLECTION',
    'FALL_R_MULTIPLIER',
    'DAILY_IMPORTS',
]

best_params_step1, lowest_loss_step1 = perform_grid_search(param_grid_step1, fixed_params_step1, param_keys_step1, train_data)

if best_params_step1:
    print(f'Best Parameters (Step 1): {dict(zip(param_keys_step1, best_params_step1))}')
else:
    print("No valid parameters found during grid search Step 1.")
print(f'Lowest Loss (Step 1): {lowest_loss_step1}')

Grid Search Progress: 100%|██████████| 3510000/3510000 [18:19:51<00:00, 53.19it/s]   


Best Parameters (Step 1): {'INITIAL_R_0': 2.9411764705882355, 'LOCKDOWN_R_0': 0.8454545454545455, 'REOPEN_R': 1.5, 'POST_REOPEN_EQUILIBRIUM_R': 0.9035714285714286, 'MORTALITY_RATE': 0.005, 'INFLECTION_DAY': datetime.date(2020, 3, 18), 'REOPEN_DATE': datetime.date(2020, 5, 20), 'REOPEN_SHIFT_DAYS': 0, 'RATE_OF_INFLECTION': 0.25, 'LOCKDOWN_FATIGUE': 1.0, 'REOPEN_INFLECTION': 0.3, 'FALL_R_MULTIPLIER': 1.001, 'DAILY_IMPORTS': 100}
Lowest Loss (Step 1): 1334092.5726309689


In [5]:
# Manually input the best values from Step 1
best_values_step1 = {
    'INITIAL_R_0': best_params_step1[0],
    'LOCKDOWN_R_0': best_params_step1[1],
    'REOPEN_R': best_params_step1[2],
    'POST_REOPEN_EQUILIBRIUM_R': best_params_step1[3],
    'MORTALITY_RATE': best_params_step1[4],
}

# Step 2: Grid search for the remaining parameters
param_grid_step2 = {
    'RATE_OF_INFLECTION': np.linspace(0.15, 0.5, 35),
    'LOCKDOWN_FATIGUE': np.linspace(0.9, 1.0, 1),
    'REOPEN_INFLECTION': np.linspace(0.15, 0.35, 20),
    'FALL_R_MULTIPLIER': np.linspace(0.9, 1.2, 300),
    'DAILY_IMPORTS': np.arange(0, 1000, 100),
}

fixed_params_step2 = list(best_values_step1.values()) + [
    INFLECTION_DAY,  # Fixed INFLECTION_DAY
    REOPEN_DATE,  # Fixed REOPEN_DATE
    0,  # Fixed REOPEN_SHIFT_DAYS
]

param_keys_step2 = list(param_grid_step2.keys()) + list(best_values_step1.keys()) + [
    'INFLECTION_DAY',
    'REOPEN_DATE',
    'REOPEN_SHIFT_DAYS',
]

best_params_step2, lowest_loss_step2 = perform_grid_search(param_grid_step2, fixed_params_step2, param_keys_step2, train_data)

if best_params_step2:
    print(f'Best Parameters (Step 2): {dict(zip(param_keys_step2, best_params_step2))}')
else:
    print("No valid parameters found during grid search Step 2.")
print(f'Lowest Loss (Step 2): {lowest_loss_step2}')

Grid Search Progress: 100%|██████████| 2100000/2100000 [10:08:39<00:00, 57.50it/s]  


Best Parameters (Step 2): {'RATE_OF_INFLECTION': 0.43823529411764706, 'LOCKDOWN_FATIGUE': 0.9, 'REOPEN_INFLECTION': 0.29736842105263156, 'FALL_R_MULTIPLIER': 1.011371237458194, 'DAILY_IMPORTS': 100, 'INITIAL_R_0': 2.9411764705882355, 'LOCKDOWN_R_0': 0.8454545454545455, 'REOPEN_R': 1.5, 'POST_REOPEN_EQUILIBRIUM_R': 0.9035714285714286, 'MORTALITY_RATE': 0.005, 'INFLECTION_DAY': datetime.date(2020, 3, 18), 'REOPEN_DATE': datetime.date(2020, 5, 20), 'REOPEN_SHIFT_DAYS': 0}
Lowest Loss (Step 2): 457196.9840432388


In [6]:
# Validate the best parameters on validation set
if best_params_step2 is not None:
    val_loss = run_simulation(best_params_step2, val_data, param_keys_step2)
    print(f'Validation Loss: {val_loss}')
else:
    print("No valid parameters found during grid search.")

Validation Loss: 980327.9624279134
